In [ ]:
# import libraries
import numpy as np
import pandas as pd
import datasets 
from datasets import Features, Image, load_dataset, load_dataset_builder, config, concatenate_datasets
import matplotlib.pyplot as plt
import os
# import shutil
# import pathlib
# import PIL
# from PIL import Image

import torch
import torchvision
from torchvision.transforms import v2, ToTensor, Lambda
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader, Subset

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# get data (from https://huggingface.co/datasets/YakupAkdin/instrument-images/tree/main)
dataset = load_dataset("YakupAkdin/instrument-images", split='train').with_format("torch")

label_values = {
    0: 'harp',
    1: 'baglama',
    2: 'electric guitar',
    3: 'acoustic guitar',
    4: 'kanun',
    5: 'violin',
    6: 'kemence',
    7: 'mandolin',
    8: 'oud',
    9: 'yayli tambur'}

inverted_labels = {v: k for k, v in label_values.items()}

In [ ]:
dataset_RGB = dataset.filter(lambda x: x['image'].dim() == 3)
len(dataset_RGB)

In [ ]:
# the RGB images are [H,W,3] and PyTorch needs them as [3,H,W]
# included [:3, :, :] slice to adjust any RGBA images to RGB
print(dataset_RGB[0]['image'].shape)

# permute the tensors
def permute_images(batch):
    # Apply the permute operation to each tensor in the batch
    batch['image'] = [img.permute(2, 0, 1)[:3, :, :] for img in batch['image']]
    return batch

dataset_RGB = dataset_RGB.map(permute_images, batched=True, batch_size=100)

dataset_RGB[0]['image'].shape

In [ ]:
dataset_greyscale = dataset.filter(lambda x: x['image'].dim() == 2)
len(dataset_greyscale)

In [ ]:
# some of the images are greyscale, so we convert to RGB as well
print(dataset_greyscale[0]['image'].shape)

def grayscale_to_rgb(item):
    item['image'] = item['image'].unsqueeze(0).repeat(3, 1, 1)
    return item

dataset_greyscale = dataset_greyscale.map(grayscale_to_rgb)
print(dataset_greyscale[0]['image'].shape)

In [ ]:
# concatenate both reformatted RGB datasets
dataset = concatenate_datasets([dataset_RGB, dataset_greyscale])

In [ ]:
# confirming 0-255 scaling of image tensors
sample_image = dataset[0]['image']

print("Sample values:", sample_image.flatten()[:10])
print("Max value:", sample_image.max().item())
print("Min value:", sample_image.min().item())

In [ ]:
# rescale tensors from 0-255 to 0-1
class ScaleTensor(object):
    def __call__(self, tensor):
        return tensor / 255.

# compose transformation function -> this is a more standard transform (according to pytorch documentation) to use later. Fitting my transform to the CIFAR tutorial model for now
# basic_transform = v2.Compose([
#     ScaleTensor(),
#     v2.RandomResizedCrop(size=(224, 224), antialias=True),
#     v2.RandomHorizontalFlip(p=0.5),
#     v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
# ])

basic_transform = v2.Compose([
    ScaleTensor(),
    v2.Resize(32),
    v2.CenterCrop(32),
    v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

# application function for transforms
def apply_transform(example):
    # Apply the transformation to the 'image' field
    example['image'] = [basic_transform(img) for img in example['image']]
    return example

# apply transforms
dataset = dataset.map(apply_transform, batched=True, batch_size=100)

### Examine Dataset/Model Compatibility Before Training

In [ ]:
# are tensor values normalized to [-1, 1]?

correct_tensor_range = 0

for i, image in enumerate(dataset['image']):
    if image.min() < -1 or image.max() > 1:
        print(f"Image at index {i} has values outside expected range: min={image.min()}, max={image.max()}")
    else:
        correct_tensor_range += 1

print(correct_tensor_range)

In [ ]:
# are all tensors shape [3, 32, 32]?

correct_tensor_shape = 0

for i in range(len(dataset)):
    if dataset[i]['image'].shape != torch.Size([3, 32, 32]):  # replace with expected shape
        print(f"Image at index {i} has incorrect shape: {image.shape}")
    else:
        correct_tensor_shape += 1

print(correct_tensor_shape)

In [ ]:
# checking images and labels for consistency
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# get some random training images
def check_instrument(instr, ds):

    check = ds.filter(lambda x: x['label'] == instr)
    images = check['image'][0:32]
    labels = check['label'][0:32]

    # show images & labels
    imshow(torchvision.utils.make_grid(images))

    label_list = ''
    for i, lbl in enumerate(labels):
        label_list += label_values[labels[i].item()]
        if i % 8 == 7:
            label_list += '\n'
        else:
            label_list += ' | '
    print(label_list)

In [ ]:
# should be all violins
check_instrument(5, dataset)

In [ ]:
# should be all electric guitars
check_instrument(2, dataset)

### Resources

https://huggingface.co/docs/datasets/use_with_pytorch
https://huggingface.co/docs/datasets/v2.14.5/en/image_load

**Process images:**
- image processing: https://huggingface.co/docs/datasets/image_process#map
- transforms methods: https://pytorch.org/vision/stable/transforms.html#transforms
- HF general processing: https://huggingface.co/docs/datasets/process
- transforms v2 reference: https://pytorch.org/vision/stable/auto_examples/transforms/plot_transforms_getting_started.html#sphx-glr-auto-examples-transforms-plot-transforms-getting-started-py

**Test/Train split**
- use this: https://huggingface.co/docs/datasets/v2.15.0/en/package_reference/main_classes#datasets.Dataset.train_test_split

**Follow rest of tutorial**
- https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

**Building a Model Basics**
- https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

### Dataloader Setup

In [ ]:
# split test/train
dataset = dataset.train_test_split(test_size=0.2, seed=1, stratify_by_column="label")

In [ ]:
batch_size = 4

trainloader = DataLoader(dataset['train'], batch_size=batch_size, shuffle=True, num_workers=2)
testloader = DataLoader(dataset['test'], batch_size=batch_size, shuffle=True, num_workers=2)

In [ ]:
# check batch sizing (should be no output)
for batch in trainloader:
    if batch['image'].shape[0] != batch_size:
        print(f"Batch {i} has incorrect batch size: {images.shape[0]}")
    if batch['label'].shape[0] != batch_size:
        print(f"Batch {i} has mismatched images and labels")

In [ ]:
# start with basic model from the tutorial

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [ ]:
# Define loss function: Classification Cross-Entropy loss and SGD with momentum.

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(25):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs = data['image']
        labels = data['label']
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0

print('Finished Training')

### Saving and Testing the Model

In [ ]:
# # save the locally-trained model in current working directory
pwd = os.getcwd()
filename = 'model v001.pth'
path = os.path.join(pwd, filename)
torch.save(net.state_dict(), path)

### Check Model Performance on Test Dataset

In [ ]:
# in case we need to load model later
# pwd = os.getcwd()
# filename = 'model v001.pth'
# path = os.path.join(pwd, filename)
# net.load_state_dict(torch.load(path))

In [ ]:
## testing a batch of images

# iterate through the dataset
dataiter = iter(testloader)
batch = next(dataiter)
images = batch['image']
labels = batch['label']

# print some test imagery and labels
imshow(torchvision.utils.make_grid(images))
actual = [label_values[labels[i].item()] for i in range(4)]
print('actual: '+', '.join(actual))

# test the model output on these images
outputs = net(images)
predicted = [label_values[i.item()] for i in torch.max(outputs, 1)[1]]
print('predicted: '+', '.join(predicted))

In [ ]:
# calculate accuracy
correct = 0
total = 0

for data in testloader:
    images = data['image']
    labels = data['label']

    prediction = net(images)
    predicted = torch.max(prediction, 1)[1]
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print(f'{correct} of {total} images categorized correctly: {100 * correct // total} %')